In [1]:
import torch
import functions
import modules
import math
from optimizer import SGD

from torch import optim
from torch import Tensor
from torch import nn

# torch.set_grad_enabled(False)

- Generates a training and a test set of 1,000 points sampled uniformly in [0, 1]2, each with a
label 0 if outside the disk of radius 1/√2π and 1 inside,
- two input units, two output units, three hidden layers of 25 units,

## Test.py

## Generate data

In [2]:
def generate_disc_set(nb):
    input = torch.Tensor(nb, 2).uniform_(0, 1)
    target = input.pow(2).sum(1).sub(1 / (2 * math.pi)).sign().add(1).div(2).long()
    return input, target

In [3]:
def convert_to_one_hot_labels(input, target):
    tmp = input.new_zeros(target.size(0), target.max() + 1)
    tmp.scatter_(1, target.view(-1, 1), 1.0)
    return tmp

In [4]:
train_input, train_target = generate_disc_set(3)

In [5]:
print(train_input)
print(train_target)

tensor([[0.3264, 0.0048],
        [0.3729, 0.4841],
        [0.1035, 0.7416]])
tensor([0, 1, 1])


In [6]:
train_target = convert_to_one_hot_labels(train_input, train_target)

In [7]:
print(train_input)
print(train_target)

tensor([[0.3264, 0.0048],
        [0.3729, 0.4841],
        [0.1035, 0.7416]])
tensor([[1., 0.],
        [0., 1.],
        [0., 1.]])


In [8]:
train_input, train_target = generate_disc_set(1000)
train_target = convert_to_one_hot_labels(train_input, train_target)

In [9]:
mean, std = train_input.mean(), train_input.std()
train_input.sub_(mean).div_(std)

tensor([[-1.3037,  0.5780],
        [-1.4725,  1.7389],
        [ 0.2799, -0.1387],
        ...,
        [ 1.3758,  0.5582],
        [-1.6316,  1.5884],
        [-0.5628, -1.5129]])

In [10]:
train_input = 0.9 * train_input

In [11]:
print(train_input.shape)
print(train_target.shape)

torch.Size([1000, 2])
torch.Size([1000, 2])


In [12]:
arch1 = modules.Sequential(modules.Linear(2, 25), modules.Linear(25, 2), modules.MSELoss(train_target))

In [13]:
for name, m in arch1.__dict__.items():
    print(name)
    print(m)

Linear0
Linear1
MSELoss2


In [14]:
nb_epochs = 2
lr = 1e-6
# optimizer = test(arch1.param(), lr=lr)

In [15]:
class TEST(object):
    def __init__(self, params):
        self.params = params
        super().__init__()
        
    def modify(self):
        lr = 1e-6
        params_tmp = []
        for (p, grad2) in self.params:
            p -= lr * grad2
            grad2 = torch.zeros(grad2.shape)
            params_tmp.append((p, grad2))
        self.params = params_tmp

In [16]:
optim = TEST(arch1.param())
for e in range(nb_epochs):
    output, loss = arch1.forward(train_input)
    grad = arch1.backward()
    optim.modify()

Forward: 
torch.Size([25, 2])
tensor([[ 7.3088e-02, -3.3690e-01],
        [-8.9737e-02, -1.7335e-01],
        [-3.8011e-01,  2.6705e-01],
        [-1.7428e-01, -1.1173e-01],
        [-5.6758e-02,  2.6857e-02],
        [-9.9570e-02,  1.9209e-01],
        [ 1.3904e-02,  1.7502e-01],
        [-4.6590e-01,  1.8319e-04],
        [-5.8383e-02,  2.4480e-01],
        [ 2.6056e-02,  3.0144e-01],
        [-7.5475e-02,  3.7267e-01],
        [ 6.0520e-01, -1.0372e-01],
        [-5.6926e-01,  2.4110e-01],
        [ 2.3501e-01, -2.9649e-01],
        [ 4.3823e-01,  5.5629e-03],
        [ 1.3442e-01, -1.4769e-01],
        [ 3.3229e-01, -2.3546e-01],
        [-6.3739e-01,  3.1140e-01],
        [ 4.6318e-02,  1.1005e-01],
        [-2.7633e-01,  3.9728e-02],
        [-2.5971e-01, -2.3154e-03],
        [-4.5361e-01, -2.0793e-01],
        [ 3.1425e-01,  4.2205e-01],
        [ 1.7323e-01, -2.7311e-01],
        [-3.4719e-02,  6.7270e-02]])
Forward: 
torch.Size([2, 25])
tensor([[ 0.2958,  0.3234, -0.0582,  0.

In [17]:
# nb_epochs = 2
# lr = 1e-3
# optimizer = SGD(arch1.param(), lr=lr)

# for e in range(nb_epochs):
#     output, loss = arch1.forward(train_input)
#     print(loss)
#     grad = arch1.backward()
#     for (p, grad2) in arch1.param():
#         p -= lr * grad2
#         grad2 = torch.zeros(grad2.shape)

In [18]:
nb_epochs = 50
lr = 1e-5
optimizer = SGD(arch1.param(), lr=lr)

In [19]:
for e in range(nb_epochs):
    output, loss = arch1.forward(train_input)
    print(loss)
    grad = arch1.backward()
    optimizer.step()
    optimizer.zero_grad()

Forward: 
torch.Size([25, 2])
tensor([[ 7.3088e-02, -3.3690e-01],
        [-8.9737e-02, -1.7335e-01],
        [-3.8011e-01,  2.6705e-01],
        [-1.7428e-01, -1.1173e-01],
        [-5.6758e-02,  2.6857e-02],
        [-9.9570e-02,  1.9209e-01],
        [ 1.3904e-02,  1.7502e-01],
        [-4.6590e-01,  1.8319e-04],
        [-5.8383e-02,  2.4480e-01],
        [ 2.6056e-02,  3.0144e-01],
        [-7.5475e-02,  3.7267e-01],
        [ 6.0520e-01, -1.0372e-01],
        [-5.6926e-01,  2.4110e-01],
        [ 2.3501e-01, -2.9649e-01],
        [ 4.3823e-01,  5.5629e-03],
        [ 1.3442e-01, -1.4769e-01],
        [ 3.3229e-01, -2.3546e-01],
        [-6.3739e-01,  3.1140e-01],
        [ 4.6318e-02,  1.1005e-01],
        [-2.7633e-01,  3.9728e-02],
        [-2.5971e-01, -2.3154e-03],
        [-4.5361e-01, -2.0793e-01],
        [ 3.1425e-01,  4.2205e-01],
        [ 1.7323e-01, -2.7311e-01],
        [-3.4719e-02,  6.7270e-02]])
Forward: 
torch.Size([2, 25])
tensor([[ 0.2958,  0.3234, -0.0582,  0.

          0.1945]])
tensor(0.2731)
Forward: 
torch.Size([25, 2])
tensor([[ 0.0604, -0.2883],
        [-0.1399, -0.2130],
        [-0.3850,  0.2385],
        [-0.2056, -0.1309],
        [-0.0524,  0.0573],
        [-0.0844,  0.1897],
        [-0.0554,  0.1549],
        [-0.4083,  0.0379],
        [-0.0439,  0.2193],
        [ 0.0076,  0.2935],
        [-0.0666,  0.3970],
        [ 0.6331, -0.1028],
        [-0.5782,  0.1976],
        [ 0.1999, -0.2825],
        [ 0.4619, -0.0473],
        [ 0.0771, -0.1843],
        [ 0.3187, -0.2427],
        [-0.6401,  0.2949],
        [-0.0224,  0.0690],
        [-0.2563,  0.0724],
        [-0.2660, -0.0457],
        [-0.4176, -0.1864],
        [ 0.3101,  0.4144],
        [ 0.1947, -0.2658],
        [ 0.0268,  0.0795]])
Forward: 
torch.Size([2, 25])
tensor([[ 0.2827,  0.3295, -0.0154,  0.2372,  0.0768, -0.1365,  0.5741, -0.3523,
         -0.2175,  0.1453,  0.0201, -0.3194, -0.0131,  0.3491, -0.4569,  0.3848,
          0.0626,  0.0342,  0.4879, -0.039

tensor(0.2342)
Forward: 
torch.Size([25, 2])
tensor([[ 0.0540, -0.2640],
        [-0.1650, -0.2328],
        [-0.3875,  0.2243],
        [-0.2212, -0.1405],
        [-0.0503,  0.0726],
        [-0.0768,  0.1886],
        [-0.0901,  0.1448],
        [-0.3795,  0.0568],
        [-0.0367,  0.2066],
        [-0.0016,  0.2895],
        [-0.0622,  0.4091],
        [ 0.6470, -0.1024],
        [-0.5827,  0.1759],
        [ 0.1824, -0.2756],
        [ 0.4737, -0.0737],
        [ 0.0485, -0.2026],
        [ 0.3119, -0.2464],
        [-0.6414,  0.2867],
        [-0.0568,  0.0484],
        [-0.2463,  0.0887],
        [-0.2691, -0.0674],
        [-0.3995, -0.1757],
        [ 0.3080,  0.4105],
        [ 0.2054, -0.2622],
        [ 0.0575,  0.0856]])
Forward: 
torch.Size([2, 25])
tensor([[ 0.2762,  0.3326,  0.0060,  0.2451,  0.0798, -0.1299,  0.5748, -0.3288,
         -0.2125,  0.1465,  0.0270, -0.3508,  0.0176,  0.3348, -0.4789,  0.3768,
          0.0439,  0.0689,  0.4865, -0.0250, -0.0606, -0.1942,

tensor([[ 0.2623,  0.3392,  0.0518,  0.2619,  0.0865, -0.1157,  0.5765, -0.2785,
         -0.2019,  0.1491,  0.0418, -0.4180,  0.0834,  0.3041, -0.5261,  0.3597,
          0.0038,  0.1433,  0.4835,  0.0055, -0.0326, -0.1490, -0.0526, -0.2302,
         -0.5165],
        [ 0.3033, -0.4639, -0.2130, -0.2569,  0.2641,  0.0454, -0.2148,  0.3462,
         -0.1307, -0.0032,  0.3154,  0.0847, -0.3732,  0.0097, -0.3611, -0.4156,
         -0.1106, -0.1248, -0.4055,  0.2835, -0.4039,  0.1212,  0.0858,  0.0323,
          0.2043]])
tensor(1.4322)
Forward: 
torch.Size([25, 2])
tensor([[ 0.0394, -0.2084],
        [-0.2224, -0.2781],
        [-0.3931,  0.1917],
        [-0.2569, -0.1624],
        [-0.0453,  0.1074],
        [-0.0595,  0.1859],
        [-0.1693,  0.1218],
        [-0.3137,  0.0999],
        [-0.0202,  0.1775],
        [-0.0227,  0.2805],
        [-0.0521,  0.4369],
        [ 0.6789, -0.1013],
        [-0.5930,  0.1263],
        [ 0.1423, -0.2596],
        [ 0.5007, -0.1340],
        [-

tensor(3.4287)
Forward: 
torch.Size([25, 2])
tensor([[ 0.0285, -0.1667],
        [-0.2654, -0.3121],
        [-0.3973,  0.1673],
        [-0.2837, -0.1789],
        [-0.0416,  0.1335],
        [-0.0465,  0.1839],
        [-0.2287,  0.1046],
        [-0.2644,  0.1322],
        [-0.0078,  0.1556],
        [-0.0385,  0.2737],
        [-0.0446,  0.4577],
        [ 0.7028, -0.1006],
        [-0.6007,  0.0890],
        [ 0.1122, -0.2477],
        [ 0.5210, -0.1793],
        [-0.0662, -0.2757],
        [ 0.2847, -0.2609],
        [-0.6467,  0.2537],
        [-0.1942, -0.0338],
        [-0.2062,  0.1540],
        [-0.2816, -0.1541],
        [-0.3274, -0.1326],
        [ 0.2998,  0.3952],
        [ 0.2483, -0.2477],
        [ 0.1804,  0.1100]])
Forward: 
torch.Size([2, 25])
tensor([[ 0.2502,  0.3449,  0.0916,  0.2765,  0.0922, -0.1034,  0.5779, -0.2349,
         -0.1926,  0.1514,  0.0547, -0.4762,  0.1404,  0.2775, -0.5670,  0.3448,
         -0.0309,  0.2078,  0.4808,  0.0320, -0.0083, -0.1098,

In [20]:
model = nn.Sequential(
            nn.Linear(2, 2),
#             nn.ReLU(),
#             nn.Linear(128, 2)
        )
criterion = nn.MSELoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3)
nb_epochs = 50

for e in range(nb_epochs):
    output = model(train_input)
#     print(output)
    loss = criterion(output, train_target.float())
    print(loss)
    model.zero_grad()
    loss.backward()
    optimizer.step()

AttributeError: 'TEST' object has no attribute 'SGD'

## Mini-Project Architecture

In [ ]:
architecture = modules.Sequential(modules.Linear(2, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 25),
                                  modules.TanH(),
                                  modules.Linear(25, 2),
                                  modules.MSELoss()
                                 )